In [1]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install jamo

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install easydict

  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6496 sha256=08bb5437070b0d29715a2b246d4a435372fc9507fb15b4df3045622d931fc472
  Stored in directory: /home/sumin/.cache/pip/wheels/fe/4e/02/c9c3154e4845bfdbf1fdf344f5a89f16dcbb4f627a908c9974
Successfully built easydict
Note: you may need to restart the kernel to use updated packages.


In [6]:
cd /home/sumin/Downloads/jit

/home/sumin/Downloads/jit


In [7]:
from pickle import FALSE
#-*- coding: utf-8 -*-
'''
Train bpe model and bpe-segment train/valid/test datasets.
e.g.,
python bpe_segment.py --jit /content/drive/MyDrive/Colab Notebooks/jit --vocab_size 4000
'''
import codecs
import os
import sentencepiece as spm

from jamo import h2j # syllable to jamo (letter)
import argparse
import easydict

def train_bpe(fpath, vocab_size):
    dir = os.path.dirname(fpath)
    train = f'--input={fpath} \
              --normalization_rule_name=identity \
              --model_prefix={dir}/bpe \
              --character_coverage=1 \
              --vocab_size={vocab_size} \
              --model_type=bpe'
    spm.SentencePieceTrainer.Train(train)

    # modify Dictionary
    lines = [line.replace("\t", " ") for line in codecs.open(f'{dir}/bpe.vocab', 'r', 'utf8').read().splitlines()[3:]]
    with codecs.open(f'{dir}/bpe.dict', 'w', 'utf8') as fout:
        fout.write("\n".join(lines))
    os.system(f'rm {dir}/bpe.vocab')

def apply_bpe(sp, sents, out_file):
    with codecs.open(out_file, 'w', 'utf8') as fout:
        fout.write("\n".join(" ".join(sp.EncodeAsPieces(sent)) for sent in sents))


if __name__ == "__main__":
    # arguments setting
    hp = easydict.EasyDict({
        "jit" : "/home/sumin/Downloads/jit",
        "vocab_size" : 4000
    })
    

    # train/valid/test
    train_je = codecs.open(f"{hp.jit}/je.train", 'r', 'utf8').read().splitlines()
    dev_je = codecs.open(f"{hp.jit}/je.dev", 'r', 'utf8').read().splitlines()
    test_je = codecs.open(f"{hp.jit}/je.test", 'r', 'utf8').read().splitlines()
    train_ko = codecs.open(f"{hp.jit}/ko.train", 'r', 'utf8').read().splitlines()
    dev_ko = codecs.open(f"{hp.jit}/ko.dev", 'r', 'utf8').read().splitlines()
    test_ko = codecs.open(f"{hp.jit}/ko.test", 'r', 'utf8').read().splitlines()

    # bpe train
    dir = 'data/{}k/bpe'.format(str(hp.vocab_size)[:-3])
    os.makedirs(dir, exist_ok=True)

    with codecs.open(f"{dir}/bpe.train", 'w', 'utf8') as fout:
        fout.write("\n".join(train_je + train_ko))
    train_bpe(f'{dir}/bpe.train', hp.vocab_size)

    # apply
    sp = spm.SentencePieceProcessor()
    sp.Load(f'{dir}/bpe.model')
    apply_bpe(sp, train_je, f'{dir}/train.je')
    apply_bpe(sp, dev_je, f'{dir}/dev.je')
    apply_bpe(sp, test_je, f'{dir}/test.je')
    apply_bpe(sp, train_ko, f'{dir}/train.ko')
    apply_bpe(sp, dev_ko, f'{dir}/dev.ko')
    apply_bpe(sp, test_ko, f'{dir}/test.ko')

In [1]:
!git clone https://github.com/pytorch/fairseq

'fairseq'에 복제합니다...
remote: Enumerating objects: 34565, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 34565 (delta 7), reused 13 (delta 6), pack-reused 34543
오브젝트를 받는 중: 100% (34565/34565), 24.08 MiB | 5.54 MiB/s, 완료.
델타를 알아내는 중: 100% (25122/25122), 완료.


In [1]:
cd fairseq

/home/sumin/fairseq


In [3]:
pip install --editable ./

Obtaining file:///home/sumin/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 442 kB 4.8 MB/s eta 0:00:01
  Using cached Cython-0.29.34-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (2.0 MB)
     |████████████████████████████████| 771 kB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 271 kB 7.8 MB/s eta 0:00:01
  Using cached numpy-1.24.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached torch-2.0.1-cp38-cp38-manylinux1_x86_64.whl (619.9 MB)
     |████████████████████████████████| 9.8 MB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 123 kB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 2.1 MB/s eta 0:00:011
     |████████████████████████████████| 77 kB 2.6 MB/s eta 0:00:011
     |████████████████████████████████| 118 kB 5.2 MB/s eta 0:00:01
     |███

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [2]:
import os
import argparse
import easydict

def prepro(src, tgt, vocab_size):
    dir = "/home/sumin/Downloads/jit/data/{}k".format(str(vocab_size)[:-3])
    trainpref = f"{dir}/bpe/train"
    destdir = f"{dir}/{src}-{tgt}-bin"

    prepro = f"fairseq-preprocess \
                  --source-lang {src} \
                  --target-lang {tgt} \
                  --trainpref {trainpref} \
                  --validpref {dir}/bpe/dev \
                  --testpref {dir}/bpe/test \
                  --srcdict {dir}/bpe/bpe.dict \
                  --tgtdict {dir}/bpe/bpe.dict \
                  --workers 8 \
                  --destdir {destdir}"

    os.system(prepro)

if __name__ == "__main__":
    
    hp = easydict.EasyDict({
        "src" : "ko",
        "tgt" : "je",
        "vocab_size" : 4000
    })

    prepro(hp.src, hp.tgt, hp.vocab_size)

In [6]:
pip install tensorboardX

     |████████████████████████████████| 114 kB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 6.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
!fairseq-train /home/sumin/Downloads/jit/data/4k/ko-je-bin \
    --arch transformer       \
    --optimizer adam \
    --lr 0.0005 \
    --label-smoothing 0.1 \
    --dropout 0.3       \
    --max-tokens 4000 \
    --stop-min-lr '1e-09' \
    --lr-scheduler inverse_sqrt       \
    --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy       \
    --max-epoch 23 \
    --warmup-updates 4000 \
    --warmup-init-lr '1e-07'    \
    --adam-betas '(0.9, 0.98)'       \
    --save-dir train/4k/ko-je-bin/ckpt  \
    --save-interval 1  \
    --cpu

2023-06-13 17:21:14 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': True, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': No

2023-06-13 17:21:14 | INFO | fairseq.tasks.translation | [ko] dictionary: 4001 types
2023-06-13 17:21:14 | INFO | fairseq.tasks.translation | [je] dictionary: 4001 types
2023-06-13 17:21:15 | INFO | fairseq_cli.train | TransformerModel(
  (encoder): TransformerEncoderBase(
    (dropout_module): FairseqDropout()
    (embed_tokens): Embedding(4001, 512, padding_idx=1)
    (embed_positions): SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayerBase(
        (self_attn): MultiheadAttention(
          (dropout_module): FairseqDropout()
          (k_proj): Linear(in_features=512, out_features=512, bias=True)
          (v_proj): Linear(in_features=512, out_features=512, bias=True)
          (q_proj): Linear(in_features=512, out_features=512, bias=True)
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout_module): Fa

In [2]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()


In [45]:
!fairseq-generate /home/sumin/Downloads/jit/data/4k/ko-je-bin \
  --path train/4k/ko-je-bin/ckpt/checkpoint_best.pt \
  --source-lang ko --target-lang je \
  --valid-subset 'valid' \
  --gen-subset 'test' \
  --beam 5 \
  --remove-bpe 'sentencepiece'\
  --results-path prediction  \
  --cpu

2023-06-13 22:41:01 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': True, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name':

2023-06-13 22:41:02 | INFO | fairseq.data.data_utils | loaded 5,000 examples from: /home/sumin/Downloads/jit/data/4k/ko-je-bin/test.ko-je.ko
2023-06-13 22:41:02 | INFO | fairseq.data.data_utils | loaded 5,000 examples from: /home/sumin/Downloads/jit/data/4k/ko-je-bin/test.ko-je.je
2023-06-13 22:41:02 | INFO | fairseq.tasks.translation | /home/sumin/Downloads/jit/data/4k/ko-je-bin test ko-je 5000 examples
2023-06-13 22:41:02 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-06-13 22:41:02 | INFO | fairseq.tasks.fairseq_task | reuse_dataloader = True
2023-06-13 22:41:02 | INFO | fairseq.tasks.fairseq_task | rebuild_batches = False
2023-06-13 22:41:02 | INFO | fairseq.tasks.fairseq_task | creating new batches for epoch 1
2023-06-13 22:51:58 | INFO | fairseq_cli.generate | NOTE: hypothesis and token scores are output in base 2
2023-06-13 22:51:58 | INFO | fairseq_cli.generate | Translated 5,000 sentences (103,549 tokens) in 642.8s (7.78 sentences/s, 161.08 tokens/s)


In [46]:
!grep '^H' prediction/generate-test.txt | cut -f3- > prediction/gen.out.sys # 예측된 문장 (H)
!grep '^T' prediction/generate-test.txt | cut -f2- > prediction/gen.out.ref # 타겟(정답) 문장 (T)

In [3]:
!fairseq-score \
--sys prediction/gen.out.sys \
--ref prediction/gen.out.ref

Namespace(ignore_case=False, order=4, ref='prediction/gen.out.ref', sacrebleu=False, sentence_bleu=False, sys='prediction/gen.out.sys')
BLEU4 = 43.60, 72.4/51.0/37.1/27.0 (BP=0.994, ratio=0.994, syslen=61233, reflen=61603)


In [3]:
!fairseq-interactive /home/sumin/Downloads/jit/data/4k/ko-je-bin \
  --path train/4k/ko-je-bin/ckpt/checkpoint_best.pt \
  --source-lang ko --target-lang je \
  --remove-bpe 'sentencepiece' \
  --beam 5 \
  --input=sample.txt > target.txt 

2023-06-14 15:32:03 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_na